# <center>遥感影像主成分分析</center>

In [1]:
from osgeo import gdal
import rasterio
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd

## 定义函数

In [2]:
def tif_bands(image_path):
    """
    计算指定TIFF文件中的波段数量。
    :param image_path: TIFF文件的路径。
    :return: 波段数量（整数）。
    """
    with rasterio.open(image_path) as src:
        num_bands = src.count
    print(num_bands)

def changeDtype(path, outpath):
    """
    改变影像的数据类型
    :param path: 影像的路径
    :param outpath: 改变后影像的输出路径
    :return: outpath
    """
    src = rasterio.open(path)
    profile = src.profile.copy()
    profile['dtype'] = 'float64'
    data = src.read().astype('float64')
    with rasterio.open(outpath, mode='w', **profile) as dst:
        dst.write(data)
    src.close()
    return outpath

def print_band_dtypes(path):
    """
    打印影像各波段的数据类型
    :param path: 影像的路径
    """
    with rasterio.open(path) as src:
        for band_idx in range(1, src.count + 1):
            band_data = src.read(indexes=band_idx)
            band_dtype = band_data.dtype.name
            print(f"Band {band_idx}: Data Type - {band_dtype}")

def perform_pca_per_band(tif_path, n_components=1):
    """
    对给定的多波段 TIFF 文件中每个波段进行独立的主成分分析（PCA），并返回每个波段的前 n_components 个主成分的特征值和贡献率。

    :param tif_path: 多波段 TIFF 文件路径
    :param n_components: 要计算的主成分数量，默认为 1
    :return: 字典，键为波段索引，值为该波段的特征值列表和贡献率列表
    """
    results = {}

    with rasterio.open(tif_path) as src:
        bands_data = src.read()
        # 获取每个波段的数据
        for band_index in range(src.count):
            band_data = bands_data[band_index,:,:]

            # 确保数据为浮点类型（假设已经转换过数据类型）
            assert band_data.dtype == np.float64, f"数据应已转换为float64 类型 (波段{band_index})"

            # 展平波段数据为一维数组
            band_data = band_data.flatten()

            # 掩膜掉 nan 值（如果存在）
            band_data = band_data[~np.isnan(band_data)]

            # 执行 PCA
            pca = PCA(n_components=n_components)
            pca.fit(band_data.reshape(-1, 1))

            # 存储特征值和贡献率
            results[band_index] = {
                'eigenvalues': pca.explained_variance_,
                'variance_ratios': pca.explained_variance_ratio_
            }
    return results

def perform_pca_per_band(tif_path, n_components=4, drop_nans=True):
    """
    对给定的多波段 TIFF 文件中所有波段进行合并，然后进行主成分分析（PCA），并返回前 n_components 个主成分的特征值和贡献率，以及对应各主成分的原始指标的权重（或称为载荷系数）
    :param tif_path: 多波段 TIFF 文件路径
    :param n_components: 要计算的主成分数量，默认为 4
    :return: 特征值、贡献率以及个指标（波段）对应各主成分的原始指标的权重（或称为载荷系数）
    """
    try:
        # 读取数据
        with rasterio.open(tif_path) as src:
            bands_data = src.read()

            # 确保数据为浮点类型（假设已经转换过数据类型）
            assert bands_data.dtype == np.float64, "数据应已转换为float64 类型"
            
            # 缺失值处理
            if drop_nans:
                panel_data = np.concatenate([bands_data[i, :, :].flatten().reshape(-1, 1) for i in range(bands_data.shape[0])], axis=1)
                mask = np.isnan(panel_data).any(axis=1)
                merged_data = panel_data[~mask, :]
            else:
                # 合并波段并替换 NaN 值为 0
                merged_data = np.concatenate([np.nan_to_num(bands_data[i, :, :].flatten().reshape(-1, 1)) for i in range(bands_data.shape[0])], axis=1)

            # 主成分分析
            pca = PCA(n_components)
            pca_data = pca.fit_transform(merged_data)

            # 获取结果
            variance_ratio = pd.DataFrame(data={'variance_ratio': pca.explained_variance_ratio_,
                                                'variance': pca.explained_variance_})
            component_matrix = pd.DataFrame(pca.components_, columns=[f"PC{i+1}" for i in range(n_components)],
                                            index=range(len(pca.components_)))

            return variance_ratio, component_matrix

    except FileNotFoundError as fnfe:
        raise FileNotFoundError(f"未找到文件: {tif_path}. {fnfe}")
    except rasterio.errors.RasterioError as re:
        raise IOError(f"打开或读取 TIFF 文件失败: {tif_path}. {re}")

## 打印波段数

In [3]:
image_path = "data/tif/resi.tif"
tif_bands(image_path)

4


## 数值类型转换

In [4]:
changeDtype('data/tif/resi.tif','data/tif/resi_float.tif')

'data/tif/resi_float.tif'

## 查看各波段数值类型

In [5]:
print_band_dtypes('data/tif/resi_float.tif')

Band 1: Data Type - float64
Band 2: Data Type - float64
Band 3: Data Type - float64
Band 4: Data Type - float64


## 主成分分析

In [6]:
result, component_matrix = perform_pca_per_band('data/tif/resi_float.tif')

In [7]:
result

,variance_ratio,variance
0,0.798909,0.031343
1,0.173561,0.006809
2,0.027350,0.001073
3,0.000180,0.000007


In [8]:
component_matrix

,PC1,PC2,PC3,PC4
0,-0.998320,0.028712,0.019752,-0.046295
1,0.051232,0.246891,-0.036035,-0.967017
2,-0.013778,-0.961297,0.114073,-0.250412
3,-0.023309,-0.118865,-0.992622,0.005407
